In [ ]:
import pandas as pd
import numpy as np
import json
cars = pd.read_csv('cars.csv')
test = pd.read_csv('test.csv')
vehi = pd.read_csv('vehicles.csv')

In [ ]:
drop_columns = ['county', 'model', 'region', 'region_url', 'id', 'url', 'vin', 'size', 'image_url', 'image_url', 'description', 'county', 'state', 'lat', 'long']
vehi.drop(columns = drop_columns, inplace=True)
vehi['paint_color'].fillna('other', inplace=True)
vehi['condition'].fillna('other', inplace=True)
vehi['type'].fillna('other', inplace=True)
vehi['drive'].fillna('other', inplace=True)
vehi.dropna(inplace=True)

In [ ]:
print('Test columns:')
print(test.columns)
print()
print('Cars columns:')
print(cars.columns)
print()
print('Vehi columns:')
print(vehi.columns)

In [ ]:
MILE_IN_KM = 1.60934

color_dict = {
    'чёрный': 'black',
    'белый': 'white',
    'серый': 'gray',
    'серебристый': 'silver',
    'синий': 'blue',
    'коричневый': 'gray',
    'красный': 'red',
    'зелёный': 'green',
    'бежевый': 'brown',
    'голубой': 'blue',
    'пурпурный': 'red',
    'золотистый': 'other',
    'фиолетовый': 'violet',
    'жёлтый': 'yellow',
    'оранжевый': 'orange',
    'розовый': 'other'
}
fuel_dict = {
    'gasoline': 'бензин',
    'diesel': 'дизель',
    'gas': 'газ',
    'hybrid': 'гибрид',
    'hybrid-petrol': 'гибрид',
    'electric': 'электро',
    'hybrid-diesel': 'гибрид',
    'other': 'другой'
}
body_dict = {
    'внедорожник 5 дв.': 'suv',
    'седан': 'sedan',
    'лифтбек': 'liftback',
    'хэтчбек 5 дв.': 'hatchback',
    'универсал 5 дв.': 'universal',
    'минивэн': 'minivan',
    'купе': 'coupe',
    'компактвэн': 'minivan',
    'хэтчбек 3 дв.': 'hatchback',
    'пикап двойная кабина': 'pickup',
    'внедорожник 3 дв.': 'suv',
    'купе-хардтоп': 'coupe',
    'кабриолет': 'cabriolet',
    'фургон': 'van',
    'родстер': 'coupe',
    'микровэн': 'minivan',
    'седан-хардтоп': 'sedan',
    'пикап одинарная кабина': 'pickup',
    'пикап полуторная кабина': 'pickup',
    'внедорожник открытый': 'suv',
    'лимузин': 'limousine',
    'тарга': 'cabriolet',
    'седан 2 дв.': 'coupe'
}
transmission_dict = {
    'автоматическая': 'automatic',
    'механическая': 'mechanical',
    'роботизированная': 'automatic',
    'вариатор': 'automatic'
}
drivetrain_dict = {
    'полный': 'all',
    'передний': 'front',
    'задний': 'rear',
    '4wd': 'all',
    'fwd': 'front',
    'other': 'other',
    'rwd': 'rear'
}
def calculate_capacity(hp):
    capacity = str(hp).split()[0]
    if capacity == 'undefined':
        return 0
    return float(capacity)


def calculate_volume_with_cylinders(cyl_string):
    a = float(cyl_string.split()[0])
    a *= 0.6
    return a
    
def eval_features(feature_string):
    feature_count = 0
    if len(feature_string) > 5:
        for i in eval(feature_string[2:-2]):
            feature_count += len(i['values'])
        return feature_count
    else:
        return 0


vehi['drive'] = vehi['drive'].apply(lambda x:drivetrain_dict[x])
vehi['fuel'] = vehi['fuel'].apply(lambda x:fuel_dict[x])
vehi = vehi[vehi['year'] > 1937]
vehi['manufacturer'] = vehi['manufacturer'].apply(lambda x: x.upper())
vehi = vehi[vehi['condition'] != 'salvage']
vehi = vehi[vehi['condition'] != 'new']
vehi = vehi[vehi['title_status'] == 'clean']
vehi.drop(columns=['condition', 'title_status'], inplace=True)
vehi['type'] = vehi['type'].apply(lambda x: x.lower())
vehi['type'] = vehi['type'].str.replace('mini-van', 'minivan')
vehi['type'] = vehi['type'].str.replace('offroad', 'suv')
vehi = vehi[(vehi['price'] > 500)&(vehi['price'] < 50001)]
vehi = vehi[vehi['cylinders'] != 'other']
vehi['cylinders'] = vehi['cylinders'].apply(calculate_volume_with_cylinders)
vehi['odometer'] = vehi['odometer']*MILE_IN_KM
vehi = vehi[(vehi['odometer'] > 500)&(vehi['odometer'] < 1600000)]
vehi = vehi[vehi['manufacturer'].isin(cars['manufacturer_name'].unique())]

cars = cars[cars['state']=='owned']
cars = cars[cars['has_warranty'] == False]
cars.dropna(inplace=True, axis=0)
cars['engine_fuel'] = cars['engine_fuel'].apply(lambda x:fuel_dict[x])
cars['manufacturer_name'] = cars['manufacturer_name'].apply(lambda x:x.upper())
cars['features_count'] = cars.iloc[:,-11:-1].sum(axis=1)
cars = cars[(cars['price_usd'] > 500)&(cars['price_usd'] < 50001)]
cars = cars[(cars['odometer_value'] > 500)&(cars['odometer_value'] < 1600000)]

test['color'] = test['color'].apply(lambda x:color_dict[x])
test['bodyType'] = test['bodyType'].apply(lambda x:body_dict[x])
test['vehicleTransmission'] = test['vehicleTransmission'].apply(lambda x:transmission_dict[x])
test['Привод'] = test['Привод'].apply(lambda x:drivetrain_dict[x])
test['productionDate'] = test['productionDate'].apply(lambda x:int(x))
test['mileage'] = test['mileage'].apply(lambda x:int(x))
test['engineDisplacement'] = test['engineDisplacement'].apply(calculate_capacity)
test['features_count'] = (test['Комплектация'].apply(eval_features)//8).apply(lambda x: x if x<10 else 9)


test = test.rename(columns={'bodyType': 'body_type',
                            'brand': 'manufacturer_name',
                            'mileage': 'odometer_value',
                            'vehicleTransmission': 'transmission',
                            'productionDate': 'year_produced',
                            'fuelType': 'engine_fuel',
                            'engineDisplacement': 'engine_capacity',
                            'Привод': 'drivetrain'
})
vehi = vehi.rename(columns={'type': 'body_type',
                            'manufacturer': 'manufacturer_name',
                            'odometer': 'odometer_value',
                            'year': 'year_produced',
                            'fuel': 'engine_fuel',
                            'drive': 'drivetrain',
                            'price': 'price_usd',
                            'paint_color': 'color',
                            'cylinders': 'engine_capacity'
})

test['year_produced'] = test['year_produced'] - 1938
cars['year_produced'] = cars['year_produced'] - 1938
vehi['year_produced'] = vehi['year_produced'] - 1938

In [ ]:
train_df = pd.concat([cars, vehi], axis=0, sort=True)
train_df['features_count'].fillna(cars['features_count'].median(), inplace=True)

In [ ]:
ITERATIONS = 1000
LR         = 0.1

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = 42,
                          depth = 6,
                          l2_leaf_reg = 1,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          #early_stopping_rounds = 500
                         )

In [ ]:
features = ['body_type',#0
            'manufacturer_name',#1
            'transmission',#2
            'year_produced',#3
            'engine_fuel',#4
            'color',#5
            'drivetrain',#6
            'engine_capacity',#7
            'features_count',#8
            'odometer_value'#9
]

X = train_df[features]
y = train_df['price_usd']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)

In [ ]:
model.fit(X_train, y_train,
         cat_features=[0,1,2,4,5,6],
         eval_set=(X_test, y_test),
         verbose_eval= 200,
         use_best_model=True,
         plot=True
         )

In [ ]:
model.get_feature_importance(prettified=True)

In [ ]:
test_pred = test[features]
pred = model.predict(test_pred)

In [ ]:
USD_RATE = 64.0817

sample_submission = pd.DataFrame()
sample_submission['price'] = pred*USD_RATE
sample_submission = sample_submission.reset_index().rename(columns={'index': 'id'})
sample_submission.to_csv(f'submission_v04.csv', index=False)
sample_submission.head()

In [ ]:
test_pred.head()

In [ ]:
random_car = ['hatchback', 'HONDA', 'automatic', 2018-1938, 'бензин', 'gray', 'front', '1.8', 2, 126089]
price = model.predict(random_car)
price*USD_RATE